In [292]:
import pandas as pd
datatypes = {'Bach': 'O',
'CRN': 'O',
'Academia': 'O',
'Materia': 'O',
'Maestro': 'O',
'Semestre': 'int64',
'Frecuencia': 'int64',
'Salón': 'int64',
'Hora': 'O',
'Dias': 'O',
'BLOQUE': 'O',
'Semana1': 'O',
'Semana2': 'O',
'Semana3': 'O',
'Semana4': 'O',
'Semana5': 'O',
'Semana6': 'O',
'Semana7': 'O',
'Semana8': 'O',
'Semana9': 'O',
'Semana10': 'O',
'Semana11': 'O',
'Semana12': 'O',
'Semana13': 'O',
'Semana14': 'O',
'Semana15': 'O',
'Semana16': 'O',
'Semana17': 'O',
'Semana18': 'O'}
df = pd.read_csv("propuesta aulas flexibles (1).csv", encoding="utf-8", dtype=datatypes)
df.columns = ['Bach','CRN','Academia','Materia','Maestro','Semestre','Frecuencia','Salón','Hora','Dias','BLOQUE','Semana1','Semana2','Semana3','Semana4','Semana5','Semana6','Semana7','Semana8','Semana9','Semana10','Semana11','Semana12','Semana13','Semana14','Semana15','Semana16','Semana17','Semana18']


In [293]:
df = df.applymap(lambda x: x.replace("  ", " ").strip() if isinstance(x, str) else x)

In [294]:
semanas = ['Semana1', 'Semana2','Semana3', 'Semana4', 'Semana5', 'Semana6', 'Semana7', 'Semana8','Semana9', 'Semana10', 'Semana11', 'Semana12', 'Semana13', 'Semana14','Semana15', 'Semana16', 'Semana17', 'Semana18']
df = df.apply(lambda x: x.fillna(df["Salón"]))


In [295]:
def transform_data2(x):
    x = x.copy()
    x["Dias"] = x["Dias"].apply(lambda x: str(x).split("//"))
    x["Hora"] = x["Hora"].apply(lambda x: str(x).split("//"))
    x = x.explode(["Dias", "Hora"])

    x["Dias"] = x["Dias"].apply(lambda x: str(x).split("-"))
    #x["Hora"] = x["Hora"].apply(lambda x: str(x).split("-"))
    x = x.explode("Dias")

    x["Dias"] = x["Dias"].apply(lambda x: x.strip())
    x["Hora"] = x["Hora"].apply(lambda x: x.strip())

    x["Dias"] = x["Dias"].replace({"L": "Lunes", "M": "Martes", "W": "Miercoles", "J": "Jueves", "V": "Viernes"}, regex=True)
    x["Dias"] = x["Dias"].astype(pd.CategoricalDtype(['Lunes', 'Martes', 'Miercoles','Jueves', 'Viernes'], ordered=True))
    x["Hora"] = x["Hora"].astype(int)

    for column in x.columns:
        if "Semana" in column:
            x["Info_"+column] = x["Materia"].astype(str) + "\n " + x[column].astype(str)
    
    x = x.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    return x.reset_index(drop=True)


In [296]:
datos = transform_data2(df)

In [297]:
materia_value = "/".join(datos[datos.duplicated(["Maestro", "Hora", "Dias"], keep=False)]["Materia"].unique().tolist())
datos.loc[datos.duplicated(["Maestro", "Hora", "Dias"], keep=False),"Materia"] = materia_value
bach_value = "/".join(datos[datos.duplicated(["Maestro", "Hora", "Dias"], keep=False)]["Bach"].unique().tolist())
datos.loc[datos.duplicated(["Maestro", "Hora", "Dias"], keep=False),"Bach"] = bach_value
crn_value = "/".join(datos[datos.duplicated(["Maestro", "Hora", "Dias"], keep=False)]["CRN"].unique().tolist())
datos.loc[datos.duplicated(["Maestro", "Hora", "Dias"], keep=False),"CRN"] = crn_value


In [287]:
datos[datos.duplicated(["Maestro", "Hora", "Dias"], keep=False)]["Materia"].unique()

array(['Ciencias Sociales II: Política Global II PD Cross list con P3/Tópicos de Humanidades (P3)'],
      dtype=object)

In [288]:
datos.drop_duplicates(["Maestro", "Hora", "Dias"], keep="first", inplace=True)

In [291]:
datos.isna().sum()

Bach               0
CRN                0
Academia           2
Materia            0
Maestro            0
Semestre           0
Frecuencia         0
Salón              0
Hora               0
Dias               0
BLOQUE             0
Semana1          110
Semana2          110
Semana3          110
Semana4          110
Semana5          110
Semana6          110
Semana7          110
Semana8          110
Semana9          110
Semana10         110
Semana11         110
Semana12         110
Semana13         110
Semana14         110
Semana15         110
Semana16         110
Semana17         110
Semana18         110
Info_Semana1       0
Info_Semana2       0
Info_Semana3       0
Info_Semana4       0
Info_Semana5       0
Info_Semana6       0
Info_Semana7       0
Info_Semana8       0
Info_Semana9       0
Info_Semana10      0
Info_Semana11      0
Info_Semana12      0
Info_Semana13      0
Info_Semana14      0
Info_Semana15      0
Info_Semana16      0
Info_Semana17      0
Info_Semana18      0
dtype: int64

In [277]:
datos.to_csv("clip_transformed2.csv", index=False)

# Extras

In [264]:
def produce_week2(data,week):
    if (data.Dias.nunique() == 5) & (data.Hora.nunique() == 5):
        output = data.pivot_table(index="Hora", columns="Dias", values=f"Info_Semana{week}", aggfunc=lambda x: "\n".join(x)).fillna(" ")
        output["Hora"] = output.index
        output = output[["Hora","Lunes", "Martes", "Miercoles", "Jueves", "Viernes"]]
        return output
    else:
        blank = pd.DataFrame(columns=pd.Categorical(["Hora","Lunes", "Martes", "Miercoles", "Jueves", "Viernes"], ordered=True), index=pd.Series(range(1,6)))
        result = data.pivot_table(index="Hora", columns="Dias", values=f"Info_Semana{week}", aggfunc=lambda x: "\n".join(x)).fillna(" ")
        output = result.combine_first(blank).fillna(" ")
        output = output[["Hora","Lunes", "Martes", "Miercoles", "Jueves", "Viernes"]]
        return output

In [265]:
disrrupt = [201, 205,301,305,401,405,11102,14002,14004,9102,10102]